# Парсер hh.ru. Сохраняет все необходимые в вакансии, в файл raw_data_hh.csv для последующей обработки.

In [1]:
import pandas as pd
import requests
import json
import re

from IPython.display import clear_output
from bs4 import BeautifulSoup
from time import sleep

In [2]:
HOST   = 'https://hh.ru'
FILTER = '/search/vacancy?st=searchVacancy&text=&specialization=1.89&specialization=1.221&specialization=1.82&specialization=1.110&specialization=1.25&specialization=1.270&specialization=1.10&specialization=1.9&salary=&currency_code=RUR&experience=doesNotMatter&employment=full&order_by=relevance&search_period=0&items_on_page=100&no_magic=true&L_save_area=true'

HEADERS = requests.utils.default_headers()
HEADERS.update({'user-agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.101 Safari/537.36',
                'accept':'*/*'})
GLOBAL_SUM = 1

In [149]:
def get_html_page(url, n_attempts=5, t_sleep=1):
    
    while n_attempts:
        try:
            r = requests.get(url, headers=HEADERS)
            
            if r.status_code != 200:
                return None
            
            html = r.content.decode()
            r.close()
            break
        
        except requests.exceptions.RequestException as e:
            n_attempts -= 1
            print(f'Sleep for {t_sleep} sec; {n_attempts} left')
            
            sleep(t_sleep)
            continue
            
    return html

In [4]:
def cleaner(string):
    string = re.sub('\n+|\t+|\r+', '', string)
    while string[0] in ' ,.':
        string = string[1:]
    
    while string[-1] == ' ':
        string = string[:-1]
    
    string = re.sub('\ +', ' ', string)
    string = re.sub('\. ', '.', string)
    string = re.sub(' \.', '.', string)
    string = re.sub(', ', ',', string)
    string = re.sub(' ,', ',', string)
    
    return string

In [5]:
def get_vacancy_info(url):
    raw_json = get_html_page(url)
    json_doc = json.loads(raw_json)
    
    key_skills   = json_doc['key_skills']
    vacancy_name = json_doc['name']
    raw_description  = json_doc['description']
    
    return key_skills, vacancy_name, raw_description

In [6]:
def professions_info(url, profession_name):
    idx_list = list()
    
    html = get_html_page(url)
    soup = BeautifulSoup(html, 'html.parser')
    dalshe = soup.find('a', attrs={'class':'bloko-button', 'data-qa':'pager-next'})
    
    while True:
        
        vacancies = soup.find_all('div', class_='vacancy-serp-item')
        
        for vacancy in vacancies:
            
            url_to_vacancy = vacancy.find('a', class_='bloko-link')['href'] 
            idx = url_to_vacancy.split('vacancy/')[1].split('?')[0]
            
            info = get_vacancy_info(f'https://api.hh.ru/vacancies/{idx}?host=hh.ru')
            
            key_skills, vacancy_name, raw_description = info 
            
            print(profession_name)
            print(idx)
            print(url_to_vacancy)
            print('-'*75)
            
            idx_list.append({
                                'idx'             : idx,
                                'profession'      : profession_name,
                                'key_skills'      : key_skills,
                                'vacancy_name'    : vacancy_name,
                                'raw_description' : raw_description,
                                'url'             : url_to_vacancy
                            })
            

        if not dalshe:
            break
            
        url_to_next_page = HOST + dalshe['href']
                
        html = get_html_page(url_to_next_page)
        soup = BeautifulSoup(html, 'html.parser')
        dalshe = soup.find('a', attrs={'class':'bloko-button', 'data-qa':'pager-next'})
    
    return idx_list

In [7]:
def get_list_of_professions(url):
    url_list = list()
    
    html = get_html_page(url)
    
    soup = BeautifulSoup(html, 'html.parser')
    professions = soup.find('ul', class_='multiple-column-list').\
                       find_all('li', class_='catalog__item')
    
    for profesion in professions:
        
        profesion_name = profesion.text
        profesion_url  = HOST + profesion.find('a')['href']
        
        url_list.append({
                            'profession_name' : profesion_name,
                            'url'             : profesion_url
                        })
    return url_list

In [8]:
def main_parse():
    idx_list = list()
    list_of_professions = get_list_of_professions(HOST + '/catalog/Informacionnye-tehnologii-Internet-Telekom')
        
    for profession in list_of_professions:
        url_to_vacancies = profession['url']
        idx_list += professions_info(url_to_vacancies, profession['profession_name'])
        
    return idx_list

In [9]:
%%time

IDX_LIST = main_parse()

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.77 µs


In [21]:
IDX_FRAME = pd.DataFrame(IDX_LIST)
IDX_FRAME = IDX_FRAME[IDX_FRAME.duplicated('idx') == False]
IDX_FRAME.reset_index(inplace=True)
IDX_FRAME.drop('index', axis=1, inplace=True)
IDX_FRAME

,idx,profession,key_skills,vacancy_name,raw_description,url
0,43977952,CRM системы,"[{'name': 'MySQL'}, {'name': 'JavaScript'}, {'...",Начинающий (Junior) PHP разработчик,<p><strong>Кого мы ищем:</strong></p> <p>Мы ищ...,https://hh.ru/vacancy/43977952
1,45266073,CRM системы,"[{'name': 'Analytical skills'}, {'name': 'Proj...",CRM Manager,<p><strong>Responsibilities:</strong></p> <ul>...,https://hh.ru/vacancy/45266073
2,45277149,CRM системы,"[{'name': 'Управление проектами'}, {'name': '1...",Программист 1С ERP,<p><strong>ООО &quot;Группа Партнер&quot; приг...,https://hh.ru/vacancy/45277149
3,45054153,CRM системы,"[{'name': 'Node.js'}, {'name': 'PostgreSQL'}, ...",Backend-разработчик на Node.Js,"<p>Хороший стек, интересные задачи, полезный п...",https://hh.ru/vacancy/45054153
4,45251420,CRM системы,"[{'name': 'SQL'}, {'name': 'Разработка техниче...",Team Lead,"<p><strong>«MeBot24» российская IT-компания, к...",https://hh.ru/vacancy/45251420
...,...,...,...,...,...,...
27812,44591970,Электронная коммерция,"[{'name': 'B2C продажи'}, {'name': 'CMS'}, {'n...",Менеджер по продажам (насосы и насосное оборуд...,<p><strong>Ищем в нашу команду Менеджера интер...,https://hh.ru/vacancy/44591970
27813,44606670,Электронная коммерция,[],Менеджер интернет-магазина автошин (удалено),<p>Требуется менеджер по продажам автошин чере...,https://hh.ru/vacancy/44606670
27814,44606713,Электронная коммерция,"[{'name': 'Закупки'}, {'name': 'Планирование п...",Руководитель интернет-магазина (детские игрушки),<p><strong>Сейчас мы ищем коллегу в нашу коман...,https://hh.ru/vacancy/44606713
27815,44630952,Электронная коммерция,"[{'name': 'Клиентоориентированность'}, {'name'...",Менеджер по работе с клиентами в интернет-магазин,<strong>Обязанности:</strong> <ul> <li>Обработ...,https://hh.ru/vacancy/44630952


In [17]:
def save_frame(frame, name, as_='xlsx'):
    if as_ == 'xlsx':
        frame.to_excel(f'Data/{name}.xlsx', index=False)
    
    if as_ == 'csv':
        frame.to_csv(f'Data/{name}.csv', index=False)
    
    print('Successfully saved!')

In [18]:
save_frame(IDX_FRAME, name='raw_data_hh', as_='csv')

Successfully saved!
